## **Section 1**

In [ ]:
import sys, tweepy
import pandas as pd
import csv
import re
import plotly.graph_objects as gr
import plotly.express as px
import statistics
import numpy as np
import tqdm
import random


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PSOSMpvt/Twitter.txt', delimiter = "\t", names=['text'])

In [ ]:
credLst=[]
for i in df.text:
  credLst.append(str(i))
for i in range(len(credLst)):
  credLst[i]=credLst[i].split()[-1]
# credLst

In [ ]:
auth=tweepy.OAuthHandler(credLst[0],credLst[1])
auth.set_access_token(credLst[3],credLst[4])
api=tweepy.API(auth)
if (not api):
    print("Authentication failed")
else:
    print("Authentication successful")

Authentication successful


### **1.**

In [ ]:
arrData=[]
twiData=api.user_timeline(screen_name="DCPWestDelhi",count=200,tweet_mode='extended')
arrData.extend(twiData)
limit=arrData[-1].id - 1
while (len(arrData)<3200):
    twiData=api.user_timeline(screen_name="DCPWestDelhi",count=200,max_id=limit,tweet_mode='extended')
    arrData.extend(twiData)
    limit=arrData[-1].id - 1

RateLimitError: ignored

In [ ]:
idData=[]
for i in arrData:
  idData.append(i.id_str)

In [ ]:
with open('/content/drive/MyDrive/PSOSMpvt/tempData.txt','w') as file:
  for items in arrData:
    file.write('%s\n' %items)

In [ ]:
with open('/content/drive/MyDrive/PSOSMpvt/dataHW2.txt','w') as file:
  for items in idData:
    file.write('%s\n' %items)

**a.**

In [ ]:
replyArr=[]
for i in arrData:
  replyArr.append(i.in_reply_to_status_id)

In [ ]:
retweetArr=[]
trackArr=[]
for i in range(len(replyArr)):
  if (i!=None):
    try:
      retweetArr.append(api.get_status(replyArr[i]))
      trackArr.append(arrData[i])
    except:
      continue

In [ ]:
with open('/content/drive/MyDrive/PSOSMpvt/replyData.txt','w') as file:
  for items in retweetArr:
    file.write('%s\n' %items)

In [ ]:
retweetText=[]
for i in retweetArr:
  retweetText.append(i.text)

In [ ]:
def fetchPii(strList):
  aadhar='[2-9]{1}[0-9]{3}(|\ |\-|\.)[0-9]{4}(|\ |\-|\.)[0-9]{4}'
  phoneNumber='((0|91|(\+91)|)(|\ |\-|\.)(\d{4})(|\ |\-|\.)(\d{1})(|\ |\-|\.)(\d{2})(|\ |\-|\.)(\d{3}))'
  email='[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
  ip='\d{1,3}(\.)\d{1,3}(\.)\d{1,3}(\.)\d{1,3}'
  pin='(\ )[1-9]{1}[0-9]{2}(|\ )[0-9]{3}(\ )'
  licensePlate='[a-zA-Z]{2}(|\ |\-|\.)[0-9][a-zA-Z0-9](|\ |\-|\.)([a-zA-Z]{2}|[[a-zA-Z]])(|\ |\-|\.)[0-9]{4}'
  checkList=[aadhar,phoneNumber,email,ip,pin,licensePlate]
  retList=[[],[],[],[],[],[]]
  for i in strList:
    for j in range(len(checkList)):
      tempArr=re.findall(checkList[j],i)
      if (len(tempArr)>0):
        retList[j].extend(tempArr)
  return retList

In [ ]:
outList=fetchPii(retweetText)
numOutList=[]
for i in outList:
  numOutList.append(len(i))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning:

Possible nested set at position 63



In [ ]:
piidf=pd.DataFrame(list(zip(numOutList,outList)), index =['aadhar', 'phone number', 'email ID', 'ip address', 'pincode', 'license plate'], columns =['# of PII detected','PII detected'])
piidf

,# of PII detected,PII detected
aadhar,6,"[(, ), (, ), (, ), (, ), (, ), (, )]"
phone number,13,"[(5000004676, , , , 5000, , 0, , 04, , 676), (..."
email ID,0,[]
ip address,0,[]
pincode,0,[]
license plate,2,"[(, , AR, ), ( , , BL, )]"


**b.**

In [ ]:
mediaArr=[]
mediaLinks=[]
for i in retweetArr:
  if 'media' in i.entities.keys():
    mediaArr.append(i.entities['media'])
    mediaLinks.append(i.entities['media'][0]['url'])

In [ ]:
for i in mediaLinks:
  print(i)

https://t.co/HA7GblRNbO
https://t.co/DQ0I4PD0Ey
https://t.co/6DZf70TpmB
https://t.co/wt1Ii1yG3L
https://t.co/TK1usM6vXA
https://t.co/6EIoyed0Ns
https://t.co/vfgcpXfL4Q
https://t.co/oZC75qu2GY
https://t.co/dsDp8WEL2U
https://t.co/uvqy9WiTse
https://t.co/JKkFrcUmNP
https://t.co/IRtsC6tCrT
https://t.co/FaEeHEku8f
https://t.co/WKnjOmsQeF
https://t.co/7zz1IMWNcE
https://t.co/KzpWEzjgev
https://t.co/Nk0Qt7ybUS
https://t.co/g772t1bwWr
https://t.co/GheAt6KOac
https://t.co/QNzGV4RYyp
https://t.co/28eJTQmsIT


### **2.**

In [ ]:
timeList=[]
for i in range(len(retweetArr)):
  lst=[]
  lst.append(retweetArr[i].created_at)
  lst.append(trackArr[i].created_at)
  timeList.append(lst)

In [ ]:
difTimeArr=[]
for i in timeList:
  difTimeArr.append(i[1]-i[0])

print(difTimeArr[0])

16:39:29


In [ ]:
print(min(difTimeArr))

0:00:00


In [ ]:
print(max(difTimeArr))

11 days, 1:40:24


In [ ]:
timeDifDf=pd.DataFrame(difTimeArr,columns=['time'])

In [ ]:
timeDifDf['time'].mean()

Timedelta('0 days 19:14:58.339130434')

In [ ]:
timeDifDf['time'].std()

Timedelta('1 days 13:54:59.178749627')

b.

In [ ]:
trackTimeArr=[]
retweetTimeArr=[]
for i in timeList:
  trackTimeArr.append(i[1])
  retweetTimeArr.append(i[0])

In [ ]:
timeDifDf['trackTime']=trackTimeArr
timeDifDf['retweetTime']=retweetTimeArr

In [ ]:
timeDifDf['time']=timeDifDf['time'].dt.total_seconds()

In [ ]:
graphTime=gr.Figure([gr.Scatter(x=timeDifDf["trackTime"],y=timeDifDf['time'])])
graphTime.show()

## **3.**

In [ ]:
tempFaltuDict={}
q3List=[]
while (len(q3List)<31):
  i=random.randint(0,len(retweetText))
  if i in tempFaltuDict:
    continue
  else:
    q3List.append(retweetText[i])
    tempFaltuDict[i]=True

for i in q3List:
  print(i)


@DCPWestDelhi Jail Road from Tilak Nagar to Hari Nagar has been taken over by shopkeepers and parking.  75-80 pct o… https://t.co/92JLDBD7Sm
@Dranandking @anubhavkhanduri @PMOIndia @News18UP @ChaubeyMeena @DCPWestDelhi for necessary action and to inform the complainant.
@varungulati84 @DCPWestDelhi @DCPWestDelhi for necessary action. Thanks for contacting Delhi Police.
Death of SI Ramlal is a tragic loss both to his family as well as Delhi Police. He was a sincere  and dedicated off… https://t.co/A0dk5qJE8u
DJ playing around A3 block and Asalatpur Village, Janakpuri (a residential area) till 12:40am. Wondering what the… https://t.co/6x7VZWZTi9
@DCPWestDelhi @ArvindKejriwal  this is how encroachment in govt land is being done brazenly...
Rama Road industrial… https://t.co/Kpk19tKlpK
Mr. SHO Inderpuri I request you to please  visit  E Block Bada Park Near  Gas Filling Shop, every night group of bo… https://t.co/UbTmeRv366
@DelhiPolice @vshankarsingh @ArvindKejriwal @lgdelhi @DCPWestDelhi

In [ ]:
with open('/content/drive/MyDrive/PSOSMpvt/q3File.txt','w') as file:
  for items in q3List:
    file.write('%s\n' %items)